# Limpieza de datos con PySpark: Data Science Job Posting on Glassdoor

Los [datos](https://tajamar365.sharepoint.com/:x:/s/3405-MasterIA2024-2025/ETYTQ0c-i6FLjM8rZ4iT1cgB6ipFAkainM-4V9M8DXsBiA?e=PeMtvh) fueron extraídos (scrapeados) del sitio web de Glassdoor y recoge los salarios de distintos puestos relacionados a Data.

### Resolver los siguientes requerimientos, para cada operación/moficación imprima como van quedadndo los cambios.

1. Cargar los datos y mostrar el esquema o la informacion de las columnas y el tip de dato de cada columna

In [0]:
df = spark.read.option("header", "true") \
               .option("delimiter", ";") \
               .option("multiline", "true") \
               .option("quote", "\"") \
               .option("escape", "\"") \
               .csv("/FileStore/Notebook Examen/Caso4/ds_jobs.csv")

In [0]:
df.display()
df.printSchema()

2. Eliminar duplicados

In [0]:
# Importar las funciones necesarias
from pyspark.sql.functions import col, count

# Agrupar por todas las columnas y contar las ocurrencias
df_duplicates = df.groupBy(df.columns).agg(count("*").alias("count"))

# Filtrar solo los duplicados
df_duplicates = df_duplicates.filter(col("count") > 1)

# Mostrar los duplicados
df_duplicates.display()


In [0]:
# Eliminar los registros duplicados basados en todas las columnas, aunque en este caso no se encuentra ninguno
df_clean = df.dropDuplicates()

# Mostrar el DataFrame limpio
df_clean.display()


3. Decidir que hacer con los datos faltantes 

In [0]:
# Importar las funciones necesarias
from pyspark.sql.functions import col, when

# Verificar si hay nulos en cada columna
null_counts = df_clean.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns])

# Mostrar el conteo de nulos por columna
null_counts.display()


In [0]:
from functools import reduce
# Filtrar las filas que tienen al menos un valor nulo
rows_with_nulls = df_clean.filter(reduce(lambda x, y: x | y, (col(c).isNull() for c in df.columns)))

# Mostrar las filas con valores nulos
rows_with_nulls.display()


In [0]:
# Eliminar las filas con cualquier valor nulo 
df_clean = df.dropna()

4. Decidir que hacer con los valores nulos

In [0]:
#hemos realizado este apartado en el punto anterior

5. ¿Cuántos registros tiene el csv?

In [0]:
# Contar el número de registros en el DataFrame
num_registros = df_clean.count()

# Mostrar el número de registros
print(f"El número de registros en el CSV es: {num_registros}")


6. Mostrar los valores únicos de `Job title` 

In [0]:
# Mostrar los valores únicos de la columna 'Job Title'
unique_job_titles = df.select("Job Title").distinct()

# Mostrar los valores únicos
unique_job_titles.show(truncate=False)


7. Remover la letra `K` de la columna `Salary Estimate` y multiplicar por 1000.

In [0]:
from pyspark.sql.functions import regexp_replace, col

# Mostrar los valores originales de 'Salary Estimate'
print("Valores originales de 'Salary Estimate':")
df_clean.select("Salary Estimate").show(truncate=False)

# Remover la letra 'K' y reemplazarla por '000'
df_clean_transformed = df_clean.withColumn("Salary Estimate", regexp_replace(col("Salary Estimate"), "K", "000"))

# Mostrar los valores transformados de 'Salary Estimate'
print("Valores transformados de 'Salary Estimate':")
df_clean_transformed.select("Salary Estimate").show(truncate=False)





8. Mostrar los valores únicos del campo `Salary Estimate`

In [0]:
# Mostrar los valores únicos de 'Salary Estimate'
unique_salary_estimates = df_clean_transformed.select("Salary Estimate").distinct()

# Mostrar los valores únicos
unique_salary_estimates.display(truncate=False)


9. Eliminar `(Glassdoor est.)` y `(Employer est.)` del campo `Salary Estimate`

In [0]:
from pyspark.sql.functions import regexp_replace, col

# Mostrar los valores originales de 'Salary Estimate'
print("Valores originales de 'Salary Estimate':")
df_clean_transformed.select("Salary Estimate").show(truncate=False)

# Remover '(Glassdoor est.)' y '(Employer est.)'
df_final = df_clean_transformed.withColumn("Salary Estimate", 
    regexp_replace(col("Salary Estimate"), r"\s*\(Glassdoor est.\)|\(Employer est.\)", ""))

# Mostrar los valores transformados de 'Salary Estimate'
print("Valores transformados de 'Salary Estimate':")
df_final.select("Salary Estimate").show(truncate=False)


10. Mostrar de mayor a menor los valores del campo `Salary Estimate`

In [0]:
from pyspark.sql.functions import regexp_replace, col, split

# Remover el símbolo '$'
df_cleaned_final = df_final.withColumn("Salary Estimate", regexp_replace(col("Salary Estimate"), r"\$", ""))

# Separar los valores del rango salarial en dos columnas nuevas
df_split = df_cleaned_final.withColumn("Salary Low", split(col("Salary Estimate"), "-").getItem(0).cast("float"))
df_split = df_split.withColumn("Salary High", split(col("Salary Estimate"), "-").getItem(1).cast("float"))

# Eliminar la columna original 'Salary Estimate'
df_final2 = df_split.drop("Salary Estimate")

# Mostrar los valores transformados
print("Valores transformados:")
df_final2.display(truncate=False)

# Ordenar y mostrar los valores de 'Salary High' de mayor a menor
df_sorted = df_final2.orderBy(col("Salary High").desc())
df_sorted.select("Salary High", "Salary Low").show(truncate=False)



11. De la columna `Job Description` quitar los saltos de linea `\n` del texto

In [0]:
from pyspark.sql.functions import regexp_replace, col

# Mostrar los valores originales de 'Job Description'
print("Valores originales de 'Job Description':")
df_final2.select("Job Description").display(truncate=False)

# Quitar los saltos de línea (\n) del texto
df_cleaned = df_final2.withColumn("Job Description", regexp_replace(col("Job Description"), r"\n", " "))

# Mostrar los valores transformados de 'Job Description'
print("Valores transformados de 'Job Description':")
df_cleaned.select("Job Description").display(truncate=False)


12. De la columna `Rating` muestre los valores unicos.

In [0]:
# Mostrar los valores únicos de la columna 'Rating'
unique_ratings = df_cleaned.select("Rating").distinct()

# Mostrar los valores únicos
unique_ratings.display(truncate=False)


13. Del campo `Rating` reemplazar los `-1.0` por `0.0`.

In [0]:
from pyspark.sql.functions import col, when

# Reemplazar los valores -1.0 por 0.0 en la columna 'Rating'
df_replaced = df_cleaned.withColumn("Rating", when(col("Rating") == -1.0, 0.0).otherwise(col("Rating")))

# Mostrar los valores únicos de la columna 'Rating' después de la transformación
print("Valores únicos de la columna 'Rating' después de la transformación:")
df_replaced.select("Rating").distinct().display(truncate=False)


14. Mostrar los valores unicos y ordenar los valores del campo `Company Name`.

In [0]:
# Obtener los valores únicos de 'Company Name' y ordenarlos
unique_company_names = df_replaced.select("Company Name").distinct().orderBy("Company Name")

# Mostrar los valores únicos y ordenados de 'Company Name'
unique_company_names.display(truncate=False)


15. Quitar todos los caracteres innecesarios que encuentres en el campo `Company Name`. Por ejemplo los saltos de linea `\n`

In [0]:
from pyspark.sql.functions import regexp_replace, col

# Mostrar los valores originales de 'Company Name'
print("Valores originales de 'Company Name':")
df_replaced.select("Company Name").show(truncate=False)

# Quitar los saltos de línea (\n) y otros caracteres innecesarios
df_cleaned = df_replaced.withColumn("Company Name", regexp_replace(col("Company Name"), r"\s+", " "))

# Mostrar los valores transformados de 'Company Name'
print("Valores transformados de 'Company Name':")
df_cleaned.select("Company Name").show(truncate=False)


16. En el campo `Location` convertir esa columna en dos: `City` y `State`. Las ciudades que tengas en `Location` asignar a la columna `City`. Lo mismo para `State`. Luego elimine la columna `Location`.

In [0]:
from pyspark.sql.functions import split, col

# Mostrar los valores originales de 'Location'
print("Valores originales de 'Location':")
df_cleaned.select("Location").show(truncate=False)

# Dividir la columna 'Location' en 'City' y 'State'
df_split = df_cleaned.withColumn("City", split(col("Location"), ", ").getItem(0))
df_split = df_split.withColumn("State", split(col("Location"), ", ").getItem(1))

# Eliminar la columna original 'Location'
df_final = df_split.drop("Location")

# Mostrar los valores transformados
print("Valores transformados:")
df_final.select("City", "State").show(truncate=False)


17. Repetir la misma lógica de la pregunta 16 pero para el campo `Headquarters`. En Headquarters dejar solo la ciudad, mientras que para el estado añadirla a una columna nueva ` Headquarter State`.

In [0]:
from pyspark.sql.functions import split, col

# Mostrar los valores originales de 'Headquarters'
print("Valores originales de 'Headquarters':")
df_final.select("Headquarters").show(truncate=False)

# Dividir la columna 'Headquarters' en 'Headquarter City' y 'Headquarter State'
df_split_hq = df_final.withColumn("Headquarter City", split(col("Headquarters"), ", ").getItem(0))
df_split_hq = df_split_hq.withColumn("Headquarter State", split(col("Headquarters"), ", ").getItem(1))

# Eliminar la columna original 'Headquarters'
df_final_hq = df_split_hq.drop("Headquarters")

# Mostrar los valores transformados
print("Valores transformados:")
df_final_hq.select("Headquarter City", "Headquarter State").show(truncate=False)


18. Muestre los valores únicos del campo `Headquarter State` 

In [0]:
# Obtener los valores únicos de 'Headquarter State'
unique_headquarter_states = df_final_hq.select("Headquarter State").distinct()

# Mostrar los valores únicos
unique_headquarter_states.show(truncate=False)


19. Mostrar valores unicos del campo `Size`.

In [0]:
# Obtener los valores únicos de 'Size'
unique_sizes = df_final_hq.select("Size").distinct()

# Mostrar los valores únicos
unique_sizes.show(truncate=False)


20. Quitar 'employee' de los registros del campo `Size`. Elimine tambien otros caracteres basura.

In [0]:
from pyspark.sql.functions import regexp_replace, col

# Mostrar los valores originales de 'Size'
print("Valores originales de 'Size':")
df_final_hq.select("Size").show(truncate=False)

# Quitar la palabra 'employees' y otros caracteres basura
df_cleaned = df_final_hq.withColumn("Size", regexp_replace(col("Size"), r"employees?|[*+]", "").cast("string"))

# Mostrar los valores transformados de 'Size'
print("Valores transformados de 'Size':")
df_cleaned.select("Size").show(truncate=False)





21. Reemplazar la palabra 'to' por '-' en todos los registros del campo `Size`. Reemplazar tambien '-1' por 'Unknown'. 

In [0]:
from pyspark.sql.functions import regexp_replace, col

# Mostrar los valores originales de 'Size'
print("Valores originales de 'Size':")
df_cleaned.select("Size").show(truncate=False)

# Reemplazar 'to' por '-' y '-1' por 'Unknown'
df_transformed = df_cleaned.withColumn("Size", regexp_replace(col("Size"), r"\bto\b", "-"))
df_transformed = df_transformed.withColumn("Size", regexp_replace(col("Size"), r"-1", "Unknown"))

# Mostrar los valores transformados de 'Size'
print("Valores transformados de 'Size':")
df_transformed.select("Size").show(truncate=False)



22. Mostrar el tipo de dato del campo `Type of ownership` y sus registros unicos.

In [0]:
# Mostrar el tipo de dato de la columna 'Type of ownership'
print("Tipo de dato de 'Type of ownership':")
df_transformed.select("Type of ownership").printSchema()

# Mostrar los registros únicos de la columna 'Type of ownership'
unique_ownership_types = df_transformed.select("Type of ownership").distinct()
print("Registros únicos de 'Type of ownership':")
unique_ownership_types.display(truncate=False)


23. Cambiar '-1' por 'Unknown' en todos los registros del campo `Type of ownership`.

In [0]:
from pyspark.sql.functions import col, when

# Reemplazar los valores -1 por 'Unknown' en la columna 'Type of ownership'
df_replaced = df_transformed.withColumn("Type of ownership", when(col("Type of ownership") == "-1", "Unknown").otherwise(col("Type of ownership")))

# Mostrar los registros únicos de 'Type of ownership' después de la transformación
print("Registros únicos de 'Type of ownership' después de la transformación:")
unique_ownership_types_after = df_replaced.select("Type of ownership").distinct()
unique_ownership_types_after.show(truncate=False)


24. Cambiar:  
-  `Company - Public` por `Public Company`  
-  `Company - Private` por `Private Company`  
-  `Private Practice / Firm` por `Private Company`  
-  `Subsidiary or Business Segment` por `Business`  
-  `College / University` por `Education`  
En todos los registros del campo `Type of ownership`.

In [0]:
from pyspark.sql.functions import col, when

# Mostrar los valores originales de 'Type of ownership'
print("Valores originales de 'Type of ownership':")
df_replaced.select("Type of ownership").distinct().show(truncate=False)

# Realizar las transformaciones en 'Type of ownership'
df_transformed = df_replaced.withColumn("Type of ownership",
    when(col("Type of ownership") == "Company - Public", "Public Company")
    .when(col("Type of ownership") == "Company - Private", "Private Company")
    .when(col("Type of ownership") == "Private Practice / Firm", "Private Company")
    .when(col("Type of ownership") == "Subsidiary or Business Segment", "Business")
    .when(col("Type of ownership") == "College / University", "Education")
    .otherwise(col("Type of ownership"))
)

# Mostrar los valores transformados de 'Type of ownership'
print("Valores transformados de 'Type of ownership':")
df_transformed.select("Type of ownership").distinct().show(truncate=False)


25. Mostrar el tipo de dato y los valores unicos del campo `Industry`.

In [0]:
# Mostrar el tipo de dato de la columna 'Industry'
print("Tipo de dato de 'Industry':")
df_transformed.select("Industry").printSchema()

# Mostrar los registros únicos de la columna 'Industry'
unique_industries = df_transformed.select("Industry").distinct()
print("Registros únicos de 'Industry':")
unique_industries.display(truncate=False)


26. En el mismo campo de `Industry` reemplazar '-1' por 'Not Available' y '&' por 'and'.  Vuelva a imprimir los valores unicos en orden alfabético.

* Mostrar el tipo de dato y los valores únicos del campo Industry

In [0]:
# Mostrar el tipo de dato de la columna 'Industry'
print("Tipo de dato de 'Industry':")
df_transformed.select("Industry").printSchema()

# Mostrar los registros únicos de la columna 'Industry' antes de la transformación
print("Registros únicos de 'Industry' antes de la transformación:")
unique_industries_before = df_transformed.select("Industry").distinct()
unique_industries_before.display(truncate=False)


* Realizar las transformaciones en el campo Industry

In [0]:
from pyspark.sql.functions import col, regexp_replace

# Reemplazar '-1' por 'Not Available' y '&' por 'and'
df_transformed1 = df_transformed.withColumn("Industry", regexp_replace(col("Industry"), r"-1", "Not Available"))
df_transformed1 = df_transformed1.withColumn("Industry", regexp_replace(col("Industry"), r"&", "and"))

# Mostrar los registros únicos de la columna 'Industry' después de la transformación
print("Registros únicos de 'Industry' después de la transformación y ordenados alfabéticamente:")
unique_industries_after = df_transformed1.select("Industry").distinct().orderBy("Industry")
unique_industries_after.show(truncate=False)


27. Para el campo `Sector`, muestre el tipo de dato y los valores únicos.

In [0]:
# Mostrar el tipo de dato de la columna 'Sector'
print("Tipo de dato de 'Sector':")
df_transformed1.select("Sector").printSchema()

# Mostrar los registros únicos de la columna 'Sector'
unique_sectors = df_transformed1.select("Sector").distinct()
print("Registros únicos de 'Sector':")
unique_sectors.show(truncate=False)


28. Aplica la misma lógica de la pregunta 26 pero sobre el campo `Sector`.

In [0]:
from pyspark.sql.functions import col, regexp_replace

# Reemplazar '-1' por 'Not Available' y '&' por 'and'
df_transformed2 = df_transformed1.withColumn("Sector", regexp_replace(col("Sector"), r"-1", "Not Available"))
df_transformed2 = df_transformed2.withColumn("Sector", regexp_replace(col("Sector"), r"&", "and"))

# Mostrar los registros únicos de la columna 'Sector' después de la transformación
print("Registros únicos de 'Sector' después de la transformación y ordenados alfabéticamente:")
unique_sectors_after = df_transformed2.select("Sector").distinct().orderBy("Sector")
unique_sectors_after.show(truncate=False)


29. Para el campo `Revenue`, muestre el tipo de dato y los valores únicos en orden ascedente.

In [0]:
# Mostrar el tipo de dato de la columna 'Revenue'
print("Tipo de dato de 'Revenue':")
df_transformed2.select("Revenue").printSchema()

# Mostrar los registros únicos de la columna 'Revenue' en orden ascendente
unique_revenues = df_transformed2.select("Revenue").distinct().orderBy("Revenue")
print("Registros únicos de 'Revenue' en orden ascendente:")
unique_revenues.display(truncate=False)


30. En el campo `Revenue`, cambiar:  
-  `-1` por `N/A`  
-  `Unknown / Non-Applicable` por `N/A`  
-  `Less than $1 million (USD)` por `Less than 1`
-  Quitar `$` y `(USD)`

In [0]:
from pyspark.sql.functions import col, regexp_replace

# Realizar las transformaciones en 'Revenue'
df_transformed3 = df_transformed2.withColumn("Revenue", regexp_replace(col("Revenue"), r"-1", "N/A"))
df_transformed3 = df_transformed3.withColumn("Revenue", regexp_replace(col("Revenue"), r"Unknown / Non-Applicable", "N/A"))
df_transformed3 = df_transformed3.withColumn("Revenue", regexp_replace(col("Revenue"), r"Less than \$1 million \(USD\)", "Less than 1"))
df_transformed3 = df_transformed3.withColumn("Revenue", regexp_replace(col("Revenue"), r"[\$\(USD\)]", ""))

# Mostrar los registros únicos de la columna 'Revenue' después de la transformación
print("Registros únicos de 'Revenue' después de la transformación y ordenados alfabéticamente:")
unique_revenues_after = df_transformed3.select("Revenue").distinct().orderBy("Revenue")
unique_revenues_after.show(truncate=False)


31. Borrar el campo `Competitors`.

* Mostrar el esquema y valores únicos antes de eliminar la columna Competitors

In [0]:
# Mostrar el esquema del DataFrame antes de eliminar la columna 'Competitors'
print("Esquema del DataFrame antes de eliminar la columna 'Competitors':")
df_transformed3.printSchema()

# Mostrar algunos registros para verificar el contenido
print("Valores del DataFrame antes de eliminar la columna 'Competitors':")
df_transformed3.display(truncate=False)


* Eliminar la columna Competitors y mostrar el esquema y valores únicos después

In [0]:
# Eliminar la columna 'Competitors'
df_dropped = df_transformed3.drop("Competitors")

# Mostrar el esquema del DataFrame después de eliminar la columna 'Competitors'
print("Esquema del DataFrame después de eliminar la columna 'Competitors':")
df_dropped.printSchema()

# Mostrar algunos registros para verificar el contenido
print("Valores del DataFrame después de eliminar la columna 'Competitors':")
df_dropped.display(truncate=False)


32. Crear tres columnas: `min_salary` (salario mínimo), `max_salary` (salario maximo) y `avg_salary` (salario promedio) a partir de los datos del campo `Salary Estimate`.

In [0]:
from pyspark.sql.functions import regexp_extract, col, lit
# Mostrar el esquema del DataFrame
#La creación de las tabklas salario minimo y maximo ya han sido creadas antes
df_dropped.printSchema()

# Calcular el salario promedio
df = df_dropped.withColumn("avg_salary", ((col("Salary Low") + col("Salary High")) / 2).cast("float"))

# Mostrar los nuevos valores del DataFrame
print("Valores del DataFrame después de agregar 'Salary Low', 'Salary High' y 'avg_salary':")
df.select("Salary Low", "Salary High", "avg_salary").show(truncate=False)



33. Mostrar los valores unicos del campo `Founded` y el tipo de dato.

In [0]:
# Mostrar el tipo de dato de la columna 'Founded'
print("Tipo de dato de 'Founded':")
df.select("Founded").printSchema()
# Mostrar los registros únicos de la columna 'Founded'
unique_founded = df.select("Founded").distinct().orderBy("Founded")
print("Registros únicos de 'Founded':")
unique_founded.show(truncate=False)


34. Reemplazar '-1' por '2024' en todos los registros del campo `Founded`.

In [0]:
from pyspark.sql.functions import col, when

# Reemplazar los valores -1 por 2024 en la columna 'Founded'
df_transformed = df.withColumn("Founded", when(col("Founded") == -1, 2024).otherwise(col("Founded")))

# Mostrar los registros únicos de la columna 'Founded' después de la transformación
print("Registros únicos de 'Founded' después de la transformación:")
unique_founded_after = df_transformed.select("Founded").distinct().orderBy("Founded")
unique_founded_after.show(truncate=False)


35. Crear una nueva columna o campo que se llame `company_age` con los datos que se deducen del campo `Founded`.

* Mostrar el tipo de dato y los valores únicos del campo Founded antes de la transformación

In [0]:
# Mostrar el tipo de dato de la columna 'Founded'
print("Tipo de dato de 'Founded':")
df_transformed.select("Founded").printSchema()

# Mostrar los registros únicos de la columna 'Founded' antes de la transformación
print("Registros únicos de 'Founded' antes de la transformación:")
unique_founded_before = df_transformed.select("Founded").distinct().orderBy("Founded")
unique_founded_before.show(truncate=False)


* Crear la nueva columna company_age

In [0]:
from pyspark.sql.functions import col

# Año actual
current_year = 2024

# Calcular la edad de la compañía
df = df_transformed.withColumn("company_age", current_year - col("Founded"))

# Mostrar los nuevos valores del DataFrame
print("Valores del DataFrame después de agregar 'company_age':")
df.select("Founded", "company_age").show(truncate=False)


36. Crear una columna o campo que se llame: `Job Type` y en cada registro debe ir Senior, Junior o NA según los datos del campo `Job Title`.  
- Cambiar 'sr' o 'senior' o 'lead' o 'principal' por `Senior` en el campo `Job Type`. No olvidar las mayúsculas.
- Cambiar 'jr' o 'jr.' o cualquier otra variante por `Junior`.  
- En cualquier otro caso distinto a los anteriores añadir NA.

* Mostrar el esquema y algunos registros del campo Job Title antes de la transformación

In [0]:
# Mostrar el esquema del DataFrame
df.printSchema()

# Mostrar algunos registros del campo 'Job Title'
print("Valores del campo 'Job Title' antes de la transformación:")
df.select("Job Title").show(truncate=False)


* Crear la nueva columna Job Type

In [0]:
from pyspark.sql.functions import col, when, lower

# Crear la columna 'Job Type' basada en las condiciones especificadas
df = df.withColumn("Job Type",
    when(lower(col("Job Title")).rlike("sr|senior|lead|principal"), "Senior")
    .when(lower(col("Job Title")).rlike("jr|jr\\."), "Junior")
    .otherwise("NA")
)

# Mostrar los nuevos valores del DataFrame
print("Valores del DataFrame después de agregar 'Job Type':")
df.select("Job Title", "Job Type").show(truncate=False)


37. Muestra los registros únicos del campo `Job Type`. 

In [0]:
# Mostrar los registros únicos de la columna 'Job Type'
unique_job_types = df.select("Job Type").distinct()
print("Registros únicos de 'Job Type':")
unique_job_types.show(truncate=False)


38. Partiendo del campo `Job Description` se extraer todas o las principales skills solicitadas por las empresas, por ejemplo: Python, Spark , Big Data. Cada Skill debe ir en una nueva columna de tipo Binaria ( 0 , 1) o Booleana (True,  False) de modo que cada skill va ser una nueva columna y si esa skill es solicitada por la empresa colocar 1 sino colocar 0. Por ejemplo:  

* Definir las habilidades a extraer

In [0]:
# Definir las principales habilidades a extraer
skills = ["Python", "Spark", "Big Data", "SQL", "Machine Learning", "AWS", "Hadoop", "Java", "Tableau"]


* Crear nuevas columnas binarias para cada habilidad

In [0]:
from pyspark.sql.functions import col, when, lower

# Crear nuevas columnas para cada habilidad
for skill in skills:
    df = df.withColumn(skill, when(lower(col("Job Description")).contains(skill.lower()), 1).otherwise(0))

# Mostrar los nuevos valores del DataFrame con las columnas de habilidades
df.select("Job Description", *skills).display(truncate=False)


Por ejemplo:  
| Job Title         | Salary Estimate | Job Description                                 | Rating | Company Name       | Size       | Founded | Type of ownership         | Industry                       | Sector                         | Same State      | company_age | Python | Excel |
|--------------------|-----------------|-------------------------------------------------|--------|--------------------|------------|---------|---------------------------|--------------------------------|--------------------------------|----------------|-------------|--------|-------|
| Sr Data Scientist | 137000-171000   | Description The Senior Data Scientist is resp... | 3.1    | Healthfirst        | 1001-5000  | 1993    | Nonprofit Organization    | Insurance Carriers            | Insurance Carriers            | Same State      | 31          | 0      | 0     |
| Data Scientist    | 137000-171000   | Secure our Nation, Ignite your Future Join th... | 4.2    | ManTech            | 5001-10000 | 1968    | Public Company            | Research and Development      | Research and Development      | Same State      | 56          | 0      | 0     |
| Data Scientist    | 137000-171000   | Overview Analysis Group is one of the larges... | 3.8    | Analysis Group      | 1001-5000  | 1981    | Private Company           | Consulting                    | Consulting                    | Same State      | 43          | 1      | 1     |
| Data Scientist    | 137000-171000   | JOB DESCRIPTION: Do you have a passion for Da... | 3.5    | INFICON            | 501-1000   | 2000    | Public Company            | Electrical and Electronic Manufacturing | Electrical and Electronic Manufacturing | Different State | 24          | 1      | 1     |


39. Exportar dataset final a csv

In [0]:
df.display()

40. Extraer todos los insights posibles que sean de valor o utilidad. Cree nuevas columnas, agrupar,  filtrar hacer varios plots que muestren dichos insights que sean de utilidad para una empresa o para un usuario. Elabore conclusiones con los insights encontrados. 

In [0]:
import matplotlib.pyplot as plt

# Calcular la media del salario promedio por tipo de trabajo
avg_salary_by_job_type = df.groupBy("Job Type").avg("avg_salary").orderBy("avg(avg_salary)", ascending=False)

# Mostrar los resultados
print("Media del salario promedio por tipo de trabajo:")
avg_salary_by_job_type.show(truncate=False)

# Convertir a pandas para visualizar
avg_salary_by_job_type_pd = avg_salary_by_job_type.toPandas()

# Graficar la media del salario promedio por tipo de trabajo
plt.figure(figsize=(10, 6))
plt.bar(avg_salary_by_job_type_pd["Job Type"], avg_salary_by_job_type_pd["avg(avg_salary)"], color=['blue', 'green', 'red'])
plt.xlabel("Job Type")
plt.ylabel("Average Salary")
plt.title("Average Salary by Job Type")
plt.show()


In [0]:
# Calcular la cantidad de trabajos que requieren cada habilidad
skills_demand = df.select(skills).groupBy().sum()

# Convertir a pandas para visualizar
skills_demand_pd = skills_demand.toPandas().transpose()
skills_demand_pd.columns = ["Count"]
skills_demand_pd = skills_demand_pd.sort_values(by="Count", ascending=False)

# Graficar la demanda de habilidades
plt.figure(figsize=(12, 8))
plt.bar(skills_demand_pd.index, skills_demand_pd["Count"], color='purple')
plt.xlabel("Skills")
plt.ylabel("Number of Job Listings")
plt.title("Demand for Different Skills")
plt.xticks(rotation=45)
plt.show()


In [0]:
# Calcular la cantidad de compañías en diferentes rangos de edad
age_bins = [0, 5, 10, 20, 50, 100, 200, 300]
df = df.withColumn("age_bin", when(col("company_age").between(0, 5), "0-5 years")
                                 .when(col("company_age").between(6, 10), "6-10 years")
                                 .when(col("company_age").between(11, 20), "11-20 years")
                                 .when(col("company_age").between(21, 50), "21-50 years")
                                 .when(col("company_age").between(51, 100), "51-100 years")
                                 .when(col("company_age").between(101, 200), "101-200 years")
                                 .otherwise("200+ years"))

# Calcular la cantidad de compañías por rango de edad
company_age_distribution = df.groupBy("age_bin").count().orderBy("count", ascending=False)

# Mostrar los resultados
print("Distribución de la edad de las compañías:")
company_age_distribution.show(truncate=False)

# Convertir a pandas para visualizar
company_age_distribution_pd = company_age_distribution.toPandas()

# Graficar la distribución de la edad de las compañías
plt.figure(figsize=(12, 6))
plt.bar(company_age_distribution_pd["age_bin"], company_age_distribution_pd["count"], color='orange')
plt.xlabel("Company Age")
plt.ylabel("Number of Companies")
plt.title("Distribution of Company Age")
plt.show()


In [0]:
# Mostrar el tipo de dato de la columna 'Industry' y algunos registros del campo 'avg_salary'
df.select("Industry", "avg_salary").show(truncate=False)
# Calcular la media del salario promedio por industria
avg_salary_by_industry = df.groupBy("Industry").avg("avg_salary").orderBy("avg(avg_salary)", ascending=False)

# Mostrar los resultados
print("Media del salario promedio por industria:")
avg_salary_by_industry.show(truncate=False)

# Convertir a pandas para visualizar
avg_salary_by_industry_pd = avg_salary_by_industry.toPandas()

# Graficar la media del salario promedio por industria
plt.figure(figsize=(14, 10))
plt.barh(avg_salary_by_industry_pd["Industry"], avg_salary_by_industry_pd["avg(avg_salary)"], color='green')
plt.xlabel("Average Salary")
plt.ylabel("Industry")
plt.title("Average Salary by Industry")
plt.show()


In [0]:
# Mostrar el tipo de dato de la columna 'Industry' y algunos registros del campo 'avg_salary'
df.select("Industry", "avg_salary").show(truncate=False)
# Calcular la media del salario promedio por industria
avg_salary_by_industry = df.groupBy("Industry").avg("avg_salary").orderBy("avg(avg_salary)", ascending=False)

# Mostrar los resultados
print("Media del salario promedio por industria:")
avg_salary_by_industry.show(truncate=False)

# Convertir a pandas para visualizar
avg_salary_by_industry_pd = avg_salary_by_industry.toPandas()

# Graficar la media del salario promedio por industria
plt.figure(figsize=(14, 10))
plt.barh(avg_salary_by_industry_pd["Industry"], avg_salary_by_industry_pd["avg(avg_salary)"], color='green')
plt.xlabel("Average Salary")
plt.ylabel("Industry")
plt.title("Average Salary by Industry")
plt.show()


In [0]:
# Mostrar el esquema del DataFrame
df.printSchema()

# Mostrar algunos registros de los campos 'Size' y 'avg_salary'
print("Valores de los campos 'Size' y 'avg_salary' antes del análisis:")
df.select("Size", "avg_salary").show(truncate=False)

# Calcular la media del salario promedio por tamaño de la empresa
avg_salary_by_size = df.groupBy("Size").avg("avg_salary").orderBy("avg(avg_salary)", ascending=False)

# Mostrar los resultados
print("Media del salario promedio por tamaño de la empresa:")
avg_salary_by_size.show(truncate=False)

# Convertir a pandas para visualizar
avg_salary_by_size_pd = avg_salary_by_size.toPandas()

# Graficar la media del salario promedio por tamaño de la empresa
plt.figure(figsize=(12, 8))
plt.bar(avg_salary_by_size_pd["Size"], avg_salary_by_size_pd["avg(avg_salary)"], color='blue')
plt.xlabel("Company Size")
plt.ylabel("Average Salary")
plt.title("Average Salary by Company Size")
plt.xticks(rotation=45)
plt.show()


In [0]:
# Mostrar el tipo de dato de la columna 'Industry' y algunos registros del campo 'avg_salary'
df.select("Industry", "avg_salary").show(truncate=False)
# Calcular la media del salario promedio por industria
avg_salary_by_industry = df.groupBy("Industry").avg("avg_salary").orderBy("avg(avg_salary)", ascending=False)

# Mostrar los resultados
print("Media del salario promedio por industria:")
avg_salary_by_industry.show(truncate=False)

# Convertir a pandas para visualizar
avg_salary_by_industry_pd = avg_salary_by_industry.toPandas()

# Graficar la media del salario promedio por industria
plt.figure(figsize=(14, 10))
plt.barh(avg_salary_by_industry_pd["Industry"], avg_salary_by_industry_pd["avg(avg_salary)"], color='green')
plt.xlabel("Average Salary")
plt.ylabel("Industry")
plt.title("Average Salary by Industry")
plt.show()
